## Basic static analysis

### Packed malwares

It must import at least **GetProcAddress** and **LoadLibraryA** in order to import the "real" libraries needed by the malware.

If the execution of the **strings** command retorns few strings then probably the bynary is packed. 

It is possible to detect the type of the packer (if this is common) with the utility **PEiD**

### PE Format


_PE Header :_
Contains several useful information about the PE such as the addresses and the dimensions of the various sections, the imports ecc..

- **IMAGE_DOS_HEADER :** useless


- **IMAGE_FILE_HEADER :** contains some basic informations such as the compiled time ecc..


- **IMAGE_OPTIONAL_HEADER :** contains the information about the type of the PE (GUI application ecc..)


- **IMAGE_SECTION_HEADER :** describe the various PE sections<br><br>

     - **virtual size :** occupied space by the section in memory
                      (IF THE SIZES OF THESE TWO SECTIONS DIFFERS A LOT, PROBABLY THE BINARY IS PACKED)
     - **raw size :** occupied space by the section on disk<br><br>
     
     - **imports :** specify the imported libraries and theyr imports method (static, dynamic, runtime)<br><br>
     
        - **static linking :** the entire library code is copied inside the PE memory space.
             Uncommon solution.
             The file dimension increase.
             The code of the library is indistinguishable from the code of the PE.<br><br>
        - **dynamic linking :** The code iof the library is not copied inside the PE but there are the information, in the PE header, about how to reach the library code in memory.
        The code is executed in another memory region (different from the region where the PE is loaded) and then the code is distinguishable from the PE one.<br><br>
        - **runtime linking :** The library is not loaded at **loading time** but at **run time**, only when the library is effectively needed by the program, through the functions LoadLibrary and GetProcAddr.
        This technique is common in the malwares but uncommon in the normal programas.<br><br>
        
  - **exports :** specifies the functions exported by the executables. These functions can be called by other executables (foundamental if the file is a dll but no so common if the file is an exe)<br><br>
  
    - **sections :**<br><br> 
        - **.text :** contains the code executed by the CPU (generally the executable code resides only here)<br><br>
        - **.rdata :** contains the informations about the imports and the exports of the PE (sometimes there are two different sections, idata and rdata, that substitute the .rdata)<br><br>
        - **.data :** contains global data accessible from the entire PE<br><br>
        - **.rsrc :** contains the resources that are not considered part of the  executable such as icons ecc...<br><br>
        - **.reloc :** contains the information needed to relocate the PE in memory<br><br>

### Interesting Functions

- **kernel32.dll :** OpenProcess, GetCurrentProcess ecc.. : processes manipulation<br><br>

- **CreateFile, WriteFile ecc.. :** files manipulation<br><br>

- **user32.dll :** SetWindowsHookEx, RegisterHotKey : they might indicates a keylogger<br><br>

- **Advapi32.dll :** Windows registries manipulation<br><br>

### Exercises

#### LAB 01

**1** - 
lab01.exe : detected by 18 antivirus

lab01.dll : detected by 18 antivirus

(FROM VIRUS TOTAL)


**2** - lab01.exe : 2010-12-19 16:16:19   

lab01.dll : 2010-12-19 16:16:38

(FROM CFFEXPLORER IN THE TAB MODIFIED ON THE MAIN PAGE)

We can say that these two files were compiled at the same time and that the exe probably uses the dll


**3** - lab01.exe : not packed (Microsoft visual studio)

lab01.dll : not packed (Microsoft visual studio)

(FROM PEID)



**4** - The imported functions from kernel32.dll pointed out that probably the malware browse the filesystem of the victim 

(FROM CFFEXPLORER IN THE IMPORT DIRECTORY TAB)

 	
**5** - from the string ".exe" we can deduce that the malware searches some executable files on the victim machine.
We can also find a reference to a dll called kerne132.dll (not kernel32.dll) and we ca assume this dll as a masked one and than a malicious one

(FROM STRINGS.EXE ON THE FILE LAB01.EXE)


**6** - we can find the functions send() and connect() to 127.26.152.13

(FROM STRINGS.EXE ON THE FILE LAB01.DLL)


**7**  - The aim of the malware is probably to create a backdoor on the victim machine (We can see the functions createProcess, connect ecc..)



#### LAB 02

**1** - detected by 36 antivirus

**2** - PEID doesn't detect nothing but by the name of the sections we can understand that the binary is packed with UPX

**3** - the malware imports from kernel32 some fuction needed for the unpacking stage

From WININET.dll it imports some functions useful to connect to the internet

**4** - the malware runs as a service and we can see some urls with the utility strings.exe

WE COULD UNPACK IT WITH UPX IN ORDER TO ANALYZE IT BETTER (upx -o <FILE_NAME> <DESTINATION_FILE>)




#### LAB 03

**1** - detected 46 antivirus

**2** - the file is packed with FSG 1.0

**3 - 4 -** we can't unpack it yet


#### LAB 04

**1** - detected 44 antivirus

**2** - the file is not packed (visual studio)

**3** - 2019-08-30 22:26:59  FAKE!!

**4** - 

**5** -

**6** - using Resource Hacker :

	1 - open the PE
	2 - Resource Hacker will identify it as binary
	3 - save the resource section as binary
	4 - open the saved file with CFFExplorer

we can notice that is a bynary with in ternet functionality



## Basic dynamic analysis

#### Sandbox 
Sandboxes created in order to run malwares in a safe and confined environment

**problems :** 

- some malwares can detect the VM (evasion)


- some malwares need some sort of user interaction, in order to be executed, (es : input from command line, cick ok on a popup ecc...) that is impossible to have in a sanbox



- some foundamental registry key are missing in some sandboxes



- if the malware is a dll probably it will have some problem in the sandbox



- problem of the long sleepp (usually the malware are ran only for a short period in a sandbox)


#### Running Malwares
- **exe** : doble click on it and no problem


- **dll** :  2 methods : 
       1 - use rundll32.exe in order to call an exported function(is possible to call it by name or by ordinal)

           ES : rundll32.exe prova.dll <function_name>
                rundll32.exe prova.dll #<function_ordinal>


        2 - modify the PE header (IMAGE_FILE_DLL = 0) and modify the extension from .dll to .exe. In this manner 
            it's possible to launch the dll as an exe but only the conde inside the dll_main is triggered
        


- **service** : install the service with rundll32.exe (es : rundll32.exe dummy.dll Install) and then make it active with net start <service_name> 


#### Useful tools

- **procmon** : monitor all activities of all processes (syscall, change in the filesystem, chenge in the registry keys, network).
It's possible to apply some filters (only display filter, all the events will be catched anyway).
It's possible to run it at boot time in order to moniutor the actions that take place at boot time.


- **process explorer** : lists all the active processes with their PIDs, CPU usage, relationship ecc...
It's possible to see the dll used by the process, the open threads and the ports on wich the process is listening.
Offers an options verify that checks the signature opf the executable on disk (useless is the process is sobstituted in memory and not on disk by the malware).
If the process is substituted in memory one of the common approach to detect it is to compare the strings of the true process with the malicious one.
Useful also when we have to analyze suspect documents (open the document and look if a new process appears in the list of process explorer).


- **regshot** : takes the snapshot of the register at a given time and compare the state with a previous one highlighting the differences


- **apateDNS** : emaulates a DNS server on the local machine in order to see the requests done by the malware on port 53 and reply to them with a custom redirection.
It is possible to emulate several domains at once on the local machine in order to deal with malwares that contact a list of domains and not only one


- **netcat**


- **wireshark**

     
- **INetSim** : emulate a lot of standard services (mail, dns, webserver) on the local machine and log all the requests received.
It runs only on a Linux environment so we have to redirect all the request made by the analysis VM to the linux VM

## IDA Pro

- loading the PE using the PE option, IDA will load it in memory using the informations found in the header, while loading the PE as binary file this will be loaded in memory in a contiguous space.
This option is useful when we have to study a malware with some shellcode encoded inside it or other particular techniques that will be invisible if the PE is loaded normally.


- it is possible to specify the position where to load the PE

### Useful windows

- **Function window** : shows all the function of the executable. The one signed with an "L" are functions belonging to a library


- **Names window** : shows all the symbols (function names ecc...) found in the executable with their addresses


- **Strings windows** : shows the string of the executable


- **Imports window** : shows the imported functions


- **Exports window** : shows the exported functions


### Cross References

We can see where a function is called or where a string is used


- **code** : we can show them with the button X on the keyboard


    - on the function prologe, shows where this function is called
    
    
    - on a target address of a jmp shows which jmp reach this address
    

- **data** : shows where the string is used


### Graph options

Define how the graph will look like

- **flow chart** : build the flow of the analyzed function


- **function call** : build the flow chart of all the functons in the program.
Useful to have a general overview of the program
                      

- **Xref to** : useful to see the path in order to reach a specific buffer


- **Xref from** : useful to see who call a specified function






### Exercises

#### LAB 5-1

**1** - 1000d02e

**2** - 100163cc

**3 - 5** - directly

**4** - pics.practicalmalwareanalysis

**5** - 23

**6** - 1

**7** - 10095b34

**8** - Initialize the remote shell

**9** - EAX is the return value of the function that checks the OS version

**10** - list the working time of the robot

**11** - list the PIDs and the names of the processes

**12** - we can rename it "detectLang"

**13** - 4 / ?

**14** - 30 sec

**15** - af, type, protocol

## Recognizing C code constructs in assembly

### Global variables vs. local variables

- **Global vars** : referenced direcly by an address in memory (es : dword_40CF60) and i the program changes the content at that address this changing will be noted by the entire program


- **Local vars** : referenced by a pointer to **ebp + offset** (es : dword ptr[ebp -4])


### Arithmetic operations

**MOV** to load the operands in the registry + the operation (**sub, add, mul** ecc...) + **mov** to store the result in memory

### If statements

**cmp** + conditional jmp (**jnz** ecc...)

<img src="images/if_stmt.png">

### Loops

- **for loops** : we have to identify the 4 main components:

        1 - initialization
        2 - increment
        3 - condition check
        4 - execution of the body
    
    <img src="images/for_stmt.png">
    
   
- **while loops** : similar to the for loops but simpler, actually the while loop doesn't have the increment part


### Function call

- **cdecl** : 

    - the parameters are pushed **from right to left** on the stack
    - the **caller** must clean the stack after the callee function is returned
    - the return value is in **EAX**
    

- **stdcall** : 

    - The same as cdecl but is **the callee** that must clean the stack before the return
    

- **fast call** : 

    - **first two** parameters passed in **ECX** and **EDX** while the others are pushed from right to left on the stack
    - the **callee** must clean the stack
    

### Switch

- **if style** : The decision are made in sequence (cmp + jz) and they jump to a label that identify the code to be executed if the condition is true. The conditions are ended by an incoditional jmp to the default case.

<img src="images/switch_if_style.png">


- **jump table** : exploits an optimization technique that avoids to do all the comparisons.
This optimization exploit the variable inside the switch as an offset to jump automatically to the correct comparison

<img src="images/switch_jump_table.png">


### Arrays

They are accessed through an index multiplied by the size of the type of the array in bytes (es : 4 in the case of an array of integer) and added to the array base address.


### Structs

Same as array but the types are heterogeneous then the size to be multiplyed with the index change


### Linked lists

The key point in order to recognize this data structures is to identify recursive pointers in the assembly (pointer to elements of the same type)

### Exercises


#### Lab 6-1

**1** - if statement, checks if the return value of the function sub_40100 (check if there is an active internet connection) is different from 0

**2** - is a printf and prints "success internet connection present!"

**3** - check if there is an active internet connection


#### Lab 6-2

**1** - check if there is an active internet connection

**2** - is a printf and prints "command parsed successfully"

**3** - initialize internett functions (internetopen),download from http://practicalmalwareanalysis.com/cc.html the file conataining the command to be executed (internetopenurl) and parse them (internetreadfile)

**4** - if nested

**5** - function imported from dll WININET

**6** - receive commands from a server


#### Lab 6-3

**1** - the difference between the previous malware is that when a command is parsed is then executed using a jump table

**2** - switch with jump table


#### Lab 6-3

**1** - 

**2** - for loop